In [27]:
import pandas as pd

crop_df = pd.read_csv("data/structured_crop_yield.csv")
weather_df = pd.read_csv("data/structured_weather.csv")

In [28]:
# finding out unique values to iddentify the missing districts inbetween the datasets
cropUniqueDistricts = crop_df['DISTRICT_NAME'].unique()
print(cropUniqueDistricts)

['Achham' 'Arghakhanchi' 'Baglung' 'Baitadi' 'Bajhang' 'Bajura' 'Banke'
 'Bara' 'Bardiya' 'Bhaktapur' 'Bhojpur' 'Chitwan' 'Dadeldhura' 'Dailekh'
 'Dang' 'Darchula' 'Dhading' 'Dhankuta' 'Dhanusa' 'Dolakha' 'Dolpa' 'Doti'
 'Gorkha' 'Gulmi' 'Humla' 'Ilam' 'Jajarkot' 'Jhapa' 'Jumla' 'Kailali'
 'Kalikot' 'Kanchanpur' 'Kapilbastu' 'Kaski' 'Kathmandu' 'Kavre' 'Khotang'
 'Lalitpur' 'Lamjung' 'Mahottari' 'Makwanpur' 'Manang' 'Morang' 'Mugu'
 'Mustang' 'Myagdi' 'Nawalparasi' 'Nuwakot' 'Okhaldhunga' 'Palpa'
 'Panchthar' 'Parbat' 'Parsa' 'Pyuthan' 'Ramechhap' 'Rasuwa' 'Rautahat'
 'Rolpa' 'Rukum' 'Rupandehi' 'Salyan' 'Sankhuwasabha' 'Saptari' 'Sarlahi'
 'Sindhuli' 'Sindhupalchok' 'Siraha' 'Solukhumbu' 'Sunsari' 'Surkhet'
 'Syangja' 'Tanahu' 'Taplejung' 'Tehrathum' 'Udayapur']


In [29]:
weatherUniqueDistricts = weather_df['DISTRICT'].unique()
print(weatherUniqueDistricts)

['Arghakhanchi' 'Baglung' 'Baitadi' 'Bajhang' 'Banke' 'Bara' 'Bardiya'
 'Bhaktapur' 'Chitwan' 'Dadeldhura' 'Dailekh' 'Dang' 'Darchula' 'Dhading'
 'Dhankuta' 'Dhanusa' 'Dolakha' 'Dolpa' 'Doti' 'Gorkha' 'Gulmi' 'Humla'
 'Ilam' 'Jhapa' 'Jumla' 'Kavre' 'Kailali' 'Kanchanpur' 'Kaski' 'Kathmandu'
 'Lalitpur' 'Lamjung' 'Mahottari' 'Makwanpur' 'Manang' 'Morang' 'Mugu'
 'Mustang' 'Myagdi' 'Nawalparasi' 'Nuwakot' 'Okhaldhunga' 'Palpa'
 'Panchthar' 'Parbat' 'Rasuwa' 'Rautahat' 'Rukum' 'Rupandehi' 'Salyan'
 'Sankhuwasabha' 'Saptari' 'Sarlahi' 'Sindhuli' 'Solukhumbu' 'Sunsari'
 'Surkhet' 'Syangja' 'Tanahu' 'Taplejung' 'Tehrathum' 'Udayapur']


In [30]:
set1 = set(cropUniqueDistricts)
set2 = set(weatherUniqueDistricts)
# districts that are missing in the weather dataset but present in the crop dataset
print(set1-set2)
# districts that are missing in the crop dataset but present in the weather dataset
# renamed dsitricts with the same convention to prevent disparity
print(set2-set1)

{'Kalikot', 'Ramechhap', 'Parsa', 'Achham', 'Siraha', 'Bajura', 'Kapilbastu', 'Jajarkot', 'Sindhupalchok', 'Khotang', 'Rolpa', 'Pyuthan', 'Bhojpur'}
set()


In [31]:
# List of missing districts and their neighboring districts
missing_districts = {
    'Bhojpur': ['Dhankuta', 'Khotang', 'Sankhuwasabha', 'Solukhumbu'],
    'Jajarkot': ['Dailekh', 'Rukum (West)', 'Dolpa', 'Kalikot'],
    'Kalikot': ['Jumla', 'Mugu', 'Dailekh', 'Jajarkot'],
    'Ramechhap': ['Dolakha', 'Sindhuli', 'Okhaldhunga', 'Kavrepalanchok'],
    'Achham': ['Bajura', 'Doti', 'Dailekh', 'Kalikot'],
    'Rolpa': ['RukumWest', 'RukumEast', 'Pyuthan', 'Dang'],
    'Sindhupalchok': ['Rasuwa', 'Kavrepalanchok', 'Dolakha', 'Nuwakot'],
    'Pyuthan': ['Rolpa', 'Gulmi', 'Arghakhanchi', 'Dang'],
    'Parsa': ['Bara', 'Rautahat', 'Chitwan', 'Makwanpur'],
    'Kapilbastu': ['Rupandehi', 'Arghakhanchi', 'Dang'],
    'Siraha': ['Saptari', 'Udayapur', 'Dhanusha'],
    'Bajura': ['Bajhang', 'Achham', 'Kalikot', 'Humla'],
    'Khotang': ['Udayapur', 'Okhaldhunga', 'Bhojpur', 'Solukhumbu']
}

# Iterate over each missing district
new_rows = []
for district, nearby_districts in missing_districts.items():
    # Get the list of years from the dataset
    years = weather_df['YEAR'].unique()
    
    for year in years:
        # Filter the data for the nearby districts and the given year
        nearby_data = weather_df[(weather_df['DISTRICT'].isin(nearby_districts)) & (weather_df['YEAR'] == year)]
        
        if not nearby_data.empty:
            # Compute the average for numeric columns and truncate to 2 decimal places
            numeric_columns = nearby_data.select_dtypes(include=['number']).columns
            averaged_values = nearby_data[numeric_columns].mean().round(2).to_dict()
            
            # Create a new row for the missing district
            new_row = {'DISTRICT': district, 'YEAR': year}
            new_row.update(averaged_values)
            new_rows.append(new_row)

# Convert new rows to a DataFrame
new_rows_df = pd.DataFrame(new_rows)

# Combine the new rows with the original dataset
complete_df = pd.concat([weather_df, new_rows_df], ignore_index=True)

# Save the updated dataset
complete_df.to_csv('complete_weather_data.csv', index=False)

print("Missing districts' data has been added and saved to 'complete_weather_data.csv'.")


Missing districts' data has been added and saved to 'complete_weather_data.csv'.


In [32]:
newWeatherDf = pd.read_csv("complete_weather_data.csv")
newWeatherUnique = newWeatherDf['DISTRICT'].unique()
set3 = set(newWeatherUnique)
print(set1-set3)

set()


In [33]:
# count number of entries per district for the crop yield data set
count = crop_df['DISTRICT_NAME'].value_counts()
# print the districts that have entries lesss than 210
print(count[count < 210])


Series([], Name: count, dtype: int64)
